## face_recognition 

In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
IMAGE_SIZE = [100, 100]
train_path = 'images'
valid_path = 'valid_image'

In [3]:
#preprocessing layer to the front of vgg16
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [4]:
#dont train existing weight
for layer in vgg.layers:
    layer.trainable =False

In [5]:
#getting number of classes
folder = glob('images/*')

In [6]:
x = Flatten()(vgg.output)
prediction = Dense(len(folder), activation=tf.nn.softmax)(x)

model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [7]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)

In [8]:
#import image from datasets using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  featurewise_center = False, #set input mean 0 over the data
                                  samplewise_center = False, #set each sample mean 0
                                  featurewise_std_normalization=False, #divide inputs by std of the datasets
                                  zca_whitening = False,
                                  rotation_range=10, #randomly rotate images in the range 0-100
                                  width_shift_range = 0.2, #randomly shift image vertically. fraction of total height.
                                  vertical_flip = False
                                  )

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('images',
                                                target_size=(100,100),
                                                batch_size=32,
                                                class_mode='categorical')

test_set = test_datagen.flow_from_directory('valid_image',
                                            target_size=(100,100),
                                            batch_size=32,
                                            class_mode='categorical')

Found 300 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [9]:
#train model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=32,
  validation_steps=len(test_set)
)

C:\Users\EGAT\AppData\Local\Temp/ipykernel_16928/432834165.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/5
32/32 [==============================] - 15s 434ms/step - loss: 0.5453 - accuracy: 0.8467 - val_loss: 0.1727 - val_accuracy: 1.0000


In [10]:
model.save('model_vgg16.h5')